# Orbit

In [ ]:
from manim import *
import numpy as np

config.media_width = "75%"

## Circular Orbit Example
The following example shows you how to animate a circular orbit without using the `MoveAlongPath` animation.


In [ ]:
%%manim -qm CircularOrbit
class CircularOrbit(Scene):

    def wrap(self,angle):
        if angle >= 2 * np.pi:
            return angle - 2 * np.pi
        elif angle <= 0:
            return angle + 2 * np.pi
        else:
            return angle

    def make_circular_orbit(self):
        # make dots for the star and planet
        self.star = Dot(color=YELLOW).scale(5)
        self.planet = Dot(color=DARK_BROWN).scale(2)
        self.period = 2                 # orbital period
        self.radius = 2                 # radius
        self.v = 2 * PI / self.period   # angular velocity

        # make a circle for the orbit
        self.circle = Circle(radius=self.radius, color=GREY_A, stroke_width=1)

        # shift the circular orbit to be centered on the star
        self.circle.move_to(self.star.get_center())

        # shift the planet to be on the orbit at an angle of 0 degrees
        self.planet.move_to(self.circle.point_at_angle(0 * DEGREES))

        # group all of the objects into a single attribute
        self.circular_orbit = VGroup(self.star, self.circle, self.planet)

    def construct(self):
        self.make_circular_orbit()
        self.f_0 = np.pi                # initial true anomaly

        def update_planet_position(mobject, dt):
            theta = self.f_0 + self.v * dt      # update angle
            theta = self.wrap(theta)
            x = self.radius * np.cos(theta)
            y = self.radius * np.sin(theta)

            mobject.move_to(np.array([-x, -y, 0]))
            self.f_0 = theta

        self.play(Create(self.circular_orbit))
        self.wait()
        self.planet.add_updater(update_planet_position)
        self.wait(5*self.period)
        self.planet.remove_updater(update_planet_position)
        self.play(FadeOut(self.circular_orbit))

## Challenge Time!
Now I want you to try to animate an *elliptical* orbit. This may not be as simple as you think! Recall that the planet
 will not travel at the same speed throughout the orbit. You may need to look at the Manim documentation and/or search
 for equations related to Keplerian orbits. The steps are:

1. Define the ellipse with a semimajor axis of 2 and an eccentricity of 0.5.
2. Create dots for the planet and star (remember that the star will be at the focus of the ellipse!)
3. Define an updater function for the planet
4. Animate the ellipse for 5 orbital periods
5. Extra challenge: rotate the orbit by PI/4 and animate 5 more orbits.

In [ ]:
%%manim -qm EllipticalOrbit
class EllipticalOrbit(Scene):
    def wrap(self,angle):
        if angle >= 2 * np.pi:
            return angle - 2 * np.pi
        elif angle <= 0:
            return angle + 2 * np.pi
        else:
            return angle

    def make_ellipse(self):
        # Define ellipse parameters
        self.ecc = 0.5                        # eccentricity
        self.a = 2                            # semimajor axis
        self.b = self.a * np.sqrt(1 - self.ecc ** 2)  # Semi-minor axis
        self.period = np.sqrt(self.a ** 3)            # orbital period
        self.v = 2 * PI / self.period                 # angular velocity

        self.star = Dot(color=YELLOW).scale(5)
        self.planet = Dot(color=DARK_BROWN).scale(2)
        self.ellipse = Ellipse(width=2 * self.a, height=2 * self.b, color=GREY_A, stroke_width=1)
        self.ellipse.move_to(self.star.get_center() + (self.a * self.ecc) * RIGHT)
        self.planet.move_to(self.ellipse.point_at_angle(0 * DEGREES))
        self.elliptical_orbit = VGroup(self.star, self.ellipse, self.planet)

    def construct(self):
        self.make_ellipse()
        self.f_0 = np.pi           # initial true anomaly
        self.rotate_angle = 0

        def update_planet_ellipse(mobject, dt):
            theta = self.f_0 + self.v * dt
            theta = self.wrap(theta)
            r = (self.a * (1 - self.ecc**2)) / (1 + self.ecc * np.cos(theta))
            x = r * np.cos(theta)
            y = r * np.sin(theta)

            mobject.move_to(np.array([-x, -y, 0]) + self.star.get_center() + self.a*LEFT)
            mobject.rotate(angle=self.rotate_angle, about_point=self.star.get_center())
            self.f_0 = theta

        self.add(self.elliptical_orbit.shift(2.5*LEFT))
        self.ellipse.shift(2.5*LEFT)

        self.planet.add_updater(update_planet_ellipse)
        self.wait(5*self.period)
        self.planet.remove_updater(update_planet_ellipse)

        self.rotate_angle = np.pi/4
        self.play(Rotate(self.elliptical_orbit,angle=self.rotate_angle, about_point=self.star.get_center()))
        self.planet.add_updater(update_planet_ellipse)
        self.wait(2*self.period)
        self.planet.remove_updater(update_planet_ellipse)
        self.wait()
